### ライブラリのインポート

In [31]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import os
import math
import optuna
import re
import random

### 計算用関数

In [32]:
# 行数を指定すると所用時間を返す関数
def knapsack(row_x, data_x,required_time_x, waiting_time_x, move_time_x, attraction_no_x):
    waiting_time_x = data_x.iloc[row_x, attraction_no_x]
    ans = required_time_x + waiting_time_x + move_time_x
    return ans


# 合計所要時間と合計人気度を算出する関数
def ride_calculation(data, ride_lst, required_times, popularity, move_time):
    # print(data)
    # print(ride_lst)
    # print(required_times)
    # print(move_time)
    total_popularity = 0
    total_time = 0
    ride_time = []
    new_ride_lst = []
    # start_time = data.index[0]
    # print("start time is:", calculate_time(start_time))
    row = 0
    try:
        c=0
        for i in ride_lst:
            # total_timeが675以上かつiが0または1の場合にループを強制終了
            # ソアリン、トイストーリーMは20:15以降DPA所持者のみ乗車可能
            if (data.index[row]) >= 675 and (i == 0 or i == 1):
                c+=1
                continue
            
            else:
                # iが8以上の場合、次のループへ(該当しないアトラクション)
                if i >= 8 or i < 0:
                    # print("不明なアトラクション：", i)
                    c+=1
                    continue
                elif c>0 and i==ride_lst[c-1]:
                    c+=1
                    continue
                else:
                    # print("-" * 40)
                    # print(calculate_time(data.index[row]))
                    ride_time.append(data.index[row])
                    # print(attractions.get(i))
                    # print("移動時間", move_time)
                    waiting_time = data.iloc[row, i]
                    # print("待ち時間:", waiting_time)
                    required_time = required_times[i]
                    # print("観賞時間:", required_time)
                    ans = knapsack(row, data, required_time, waiting_time, move_time, i)
                    # print("合計所要時間:", ans)
                    total_time += ans
                    row = int(np.ceil(total_time / 15))
                    total_popularity += popularity[i]
                    new_ride_lst.append(i)
                    c+=1
    except Exception as e:
        # print("全てのアトラクションを乗ることができません")
        pass

    # print("-" * 40)
    # print(total_popularity)
    return total_popularity, total_time, new_ride_lst, ride_time

### 最適化実行関数

In [33]:
def optimization(file_pathes):
    best_plan_lst = []
    file_path = file_pathes

    # 目的関数の定義
    def objective(trial):
        # アトラクションの所要時間と人気度 or レビュー評価
        required_times = [5, 7, 2, 3, 3, 2, 23, 30]  # 所要時間
        popularity = [476, 465, 473, 472, 480, 452, 461, 478] # レビュー評価
        #popularity = [84, 69, 55, 45, 33, 28, 19, 22]  # 人気度
        move_time = 15
        
        # print(file_path)
        data = pd.read_csv(file_path)
        data = data.set_index('時間')
        
        # 乗り物の数を決定
        n_rides = trial.suggest_int('n_rides', 8, 20)
    
        # まず0~7をすべて含む部分を作る
        ride_lst = list(range(8))
    
        # もしn_ridesが8を超える場合、残りの要素をランダムに追加
        if n_rides > 8:
            ride_lst += [trial.suggest_int(f'ride_{i}', 0, 7) for i in range(n_rides - 8)]
    
        # リストをシャッフルして順番をランダムに
        random.shuffle(ride_lst)        
    
        # ride_calculation関数の呼び出し
        total_popularity, total_time, new_ride_lst, ride_time = ride_calculation(data, ride_lst, required_times, popularity, move_time)
        
        # 制約条件の範囲
        #max_allowed_time = 660  # パーク内にいる時間 9:00~20:00 計660分 (20時以降はファストパスのみ受け入れと仮定)
        max_allowed_time = 675  # パーク内にいる時間 9:00~20:15 計675分 (20:15以降はファストパスのみ受け入れと仮定)
        
        # 制約を満たす場合のみ最適化対象とする
        if total_time <= max_allowed_time:
            
            # 正規表現で年月日部分を抽出
            match = re.search(r'\d{4}-\d{2}-\d{2}', file_path)
            date = match.group() if match else None
            
            best_plan_lst.append([trial.number, date, total_popularity, total_time, ride_lst, new_ride_lst, ride_time])
            # print(total_popularity)
            return total_popularity
        else:
            # 制約を満たさない場合はペナルティを課す
            return float('-inf')

    # Optunaによる最適化
    study = optuna.create_study(direction="maximize")  # 人気度を最大化
    study.optimize(objective, n_trials = 1000)

    # 結果の表示
    best_trial = study.best_trial
    print(f"Best trial: {best_trial.number}")
    print(f"Best popularity: {best_trial.value}")
    print(f"Parameters: {best_trial.params}")
    
    # DataFrameに変換
    best_plan_df = pd.DataFrame(best_plan_lst)
    
    # best_trial.numberに対応する行を抽出
    best_plan = best_plan_df[best_plan_df[0] == best_trial.number].iloc[0]
    
    return best_plan

In [34]:
columns = ["日付", "トータル満足度", "トータル所要時間", "予定プラン", "最適プラン", "タイムスケジュール"]
df_best_csv = pd.DataFrame(columns=columns)


import time
# 処理時間の計算
start_time = time.time()

# 最適化の実行
#folder_path = 'date_data2/'
folder_path = '0724/'
best_csv = []
file_names = sorted(os.listdir(folder_path))  # フォルダ名をソート
for file_name in file_names:
    file_path = os.path.join(folder_path, file_name)
    if file_path.endswith('.csv'):
        for i in range(3): # ファイルにつき何回最適化を行うか
            plan = optimization(file_path)
            plan_data = plan.iloc[1:]
            best_csv.append(plan_data.values.tolist())
            # You can add additional logic to handle the results of each optimization run here

# best_csvリストをDataFrameに変換
columns = ["日付", "トータル満足度", "トータル所要時間", "予定プラン", "最適プラン", "タイムスケジュール"]
df_best_csv = pd.DataFrame(best_csv, columns=columns)

# 日付をインデックスに設定
df_best_csv = df_best_csv.set_index("日付")

# DataFrameをCSVファイルとして保存
df_best_csv.to_csv('best_results_0724_r3_t1000_review.csv', index=True)



end_time = time.time()
# 処理時間を計算
elapsed_time = end_time - start_time
# 処理時間を表示
print(f"処理時間: {elapsed_time} 秒")

[I 2024-10-02 18:01:25,646] A new study created in memory with name: no-name-871ecb6f-28f9-483b-9ce0-aeddb562d6d0
[I 2024-10-02 18:01:25,650] Trial 0 finished with value: -inf and parameters: {'n_rides': 14, 'ride_0': 2, 'ride_1': 2, 'ride_2': 3, 'ride_3': 6, 'ride_4': 0, 'ride_5': 2}. Best is trial 0 with value: -inf.
[I 2024-10-02 18:01:25,653] Trial 1 finished with value: 4230.0 and parameters: {'n_rides': 9, 'ride_0': 2}. Best is trial 1 with value: 4230.0.
[I 2024-10-02 18:01:25,657] Trial 2 finished with value: -inf and parameters: {'n_rides': 19, 'ride_0': 7, 'ride_1': 1, 'ride_2': 7, 'ride_3': 2, 'ride_4': 3, 'ride_5': 7, 'ride_6': 4, 'ride_7': 3, 'ride_8': 5, 'ride_9': 7, 'ride_10': 4}. Best is trial 1 with value: 4230.0.
[I 2024-10-02 18:01:25,660] Trial 3 finished with value: -inf and parameters: {'n_rides': 10, 'ride_0': 6, 'ride_1': 1}. Best is trial 1 with value: 4230.0.
[I 2024-10-02 18:01:25,664] Trial 4 finished with value: -inf and parameters: {'n_rides': 13, 'ride_0'

Best trial: 997
Best popularity: 6101.0
Parameters: {'n_rides': 13, 'ride_0': 7, 'ride_1': 5, 'ride_2': 4, 'ride_3': 6, 'ride_4': 2}


[I 2024-10-02 18:23:29,023] Trial 16 finished with value: 4690.0 and parameters: {'n_rides': 10, 'ride_0': 3, 'ride_1': 6}. Best is trial 12 with value: 5188.0.
[I 2024-10-02 18:23:29,045] Trial 17 finished with value: -inf and parameters: {'n_rides': 13, 'ride_0': 1, 'ride_1': 7, 'ride_2': 4, 'ride_3': 5, 'ride_4': 3}. Best is trial 12 with value: 5188.0.
[I 2024-10-02 18:23:29,050] Trial 18 finished with value: 3757.0 and parameters: {'n_rides': 8}. Best is trial 12 with value: 5188.0.
[I 2024-10-02 18:23:29,083] Trial 19 finished with value: -inf and parameters: {'n_rides': 18, 'ride_0': 4, 'ride_1': 5, 'ride_2': 5, 'ride_3': 3, 'ride_4': 5, 'ride_5': 0, 'ride_6': 7, 'ride_7': 7, 'ride_8': 0, 'ride_9': 3}. Best is trial 12 with value: 5188.0.
[I 2024-10-02 18:23:29,098] Trial 20 finished with value: -inf and parameters: {'n_rides': 11, 'ride_0': 1, 'ride_1': 6, 'ride_2': 2}. Best is trial 12 with value: 5188.0.
[I 2024-10-02 18:23:29,113] Trial 21 finished with value: 4235.0 and par

Best trial: 560
Best popularity: 6122.0
Parameters: {'n_rides': 14, 'ride_0': 4, 'ride_1': 5, 'ride_2': 4, 'ride_3': 4, 'ride_4': 4, 'ride_5': 2}


[I 2024-10-02 18:25:54,137] Trial 17 finished with value: 4707.0 and parameters: {'n_rides': 10, 'ride_0': 7, 'ride_1': 3}. Best is trial 11 with value: 5163.0.
[I 2024-10-02 18:25:54,149] Trial 18 finished with value: 4209.0 and parameters: {'n_rides': 10, 'ride_0': 5, 'ride_1': 2}. Best is trial 11 with value: 5163.0.
[I 2024-10-02 18:25:54,174] Trial 19 finished with value: -inf and parameters: {'n_rides': 14, 'ride_0': 2, 'ride_1': 3, 'ride_2': 4, 'ride_3': 3, 'ride_4': 4, 'ride_5': 3}. Best is trial 11 with value: 5163.0.
[I 2024-10-02 18:25:54,186] Trial 20 finished with value: 4674.0 and parameters: {'n_rides': 10, 'ride_0': 5, 'ride_1': 1}. Best is trial 11 with value: 5163.0.
[I 2024-10-02 18:25:54,229] Trial 21 finished with value: 5167.0 and parameters: {'n_rides': 12, 'ride_0': 7, 'ride_1': 4, 'ride_2': 1, 'ride_3': 5}. Best is trial 21 with value: 5167.0.
[I 2024-10-02 18:25:54,255] Trial 22 finished with value: -inf and parameters: {'n_rides': 13, 'ride_0': 7, 'ride_1': 4

Best trial: 820
Best popularity: 6138.0
Parameters: {'n_rides': 14, 'ride_0': 4, 'ride_1': 4, 'ride_2': 4, 'ride_3': 4, 'ride_4': 4, 'ride_5': 6}
処理時間: 4523.061896085739 秒


In [35]:
print(f"処理時間: {elapsed_time} 秒")

処理時間: 4523.061896085739 秒


In [36]:
df = pd.read_csv("best_results_0724_r3_t1000_review.csv")
display(df)

,日付,トータル満足度,トータル所要時間,予定プラン,最適プラン,タイムスケジュール
0,2023-07-24,6101,670,"[0, 2, 5, 4, 7, 2, 1, 6, 5, 3, 6, 4, 7]","[0, 2, 5, 4, 7, 2, 1, 6, 5, 3, 6, 4, 7]","[0, 30, 90, 135, 195, 255, 315, 420, 465, 510,..."
1,2023-07-24,6122,658,"[0, 4, 2, 6, 7, 4, 5, 1, 2, 3, 4, 4, 5, 4]","[0, 4, 2, 6, 7, 4, 5, 1, 2, 3, 4, 5, 4]","[0, 30, 75, 120, 195, 255, 315, 345, 435, 495,..."
2,2023-07-24,6138,670,"[0, 4, 4, 6, 4, 7, 2, 4, 1, 5, 3, 4, 6, 4]","[0, 4, 6, 4, 7, 2, 4, 1, 5, 3, 4, 6, 4]","[0, 30, 75, 120, 180, 240, 300, 360, 465, 495,..."
